In [1]:
import os
import re
import pandas as pd
import numpy as np

In [2]:
def change_dtype_ser(ser):
    
    if ser.dtype == int:
        return ser.astype(np.int32)
    
    if ser.dtype == float:
        return ser.astype(np.float32)
    
    if ser.dtype == np.object:
        return ser.astype("category")
    
    return ser
    

def change_dtype_df(df):
    """
    change types of columns to reduce memory size
    :param df: dataframe
    :return df: dataframe
    """
    df["fecha_dato"] = pd.to_datetime(df["fecha_dato"])
    df["fecha_alta"] = pd.to_datetime(df["fecha_alta"])
    
    memory = df.memory_usage().sum() / 10**6
    print("Memory usage before changing types %0.2f MB" % memory)

    for col in df.columns:
        df[col] = change_dtype_ser(df[col])

    memory = df.memory_usage().sum() / 10 ** 6
    print("Memory usage after changing types %0.2f MB" % memory)
    return df


def load_csv(filename):
    df = pd.read_csv(filename)
    df = change_dtype_df(df)
    return df

In [3]:
INP_DIR = "data/data_"
OUT_DIR1 = "data/data1_"
OUT_DIR2 = "data/data2_"
OUT_DIR3 = "data/data3_"

In [4]:
df_train = load_csv(os.path.join(INP_DIR, "train_cleaned.csv"))
df_test = load_csv(os.path.join(INP_DIR, "test_cleaned.csv"))

Memory usage before changing types 10044.42 MB
Memory usage after changing types 4858.46 MB
Memory usage before changing types 148.74 MB
Memory usage after changing types 63.23 MB


In [5]:
# drop this column becuase it is too imbalanced
df_train = df_train.drop(["ind_empleado"], axis=1)
df_test = df_test.drop(["ind_empleado"], axis=1)

In [6]:
# days of joining since the month data is recorded 
df_train["fecha_alta"] = (df_train["fecha_alta"] - df_train["fecha_dato"]).dt.days
df_test["fecha_alta"] = (df_test["fecha_alta"] - df_test["fecha_dato"]).dt.days

In [7]:
df_train.isnull().sum().sum(), df_test.isnull().sum().sum()

(0, 0)

In [8]:
df_train.shape, df_test.shape

((13647309, 91), (929615, 19))

In [9]:
df_train.columns

Index(['fecha_dato', 'ncodpers', 'pais_residencia', 'sexo', 'age',
       'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall',
       'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1',
       'ind_ahor_fin_ult1_NEW_PUR', 'ind_aval_fin_ult1_NEW_PUR',
       'ind_cco_fin_ult1_NEW_PUR', 'ind_cder_fin_ult1_NEW_PUR',
       'ind_cno_fin_ult1_NEW_PUR', 'ind_ctj

In [10]:
df_test.columns

Index(['fecha_dato', 'ncodpers', 'pais_residencia', 'sexo', 'age',
       'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall',
       'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento'],
      dtype='object')

In [11]:
PROD_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1$", col)]
print(PROD_COLS)

['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']


In [12]:
# total number of prod
df_train["TOTAL_PRODS"] = df_train[PROD_COLS].sum(axis=1)

In [13]:
NEW_PUR_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1_NEW_PUR$", col)]
print(NEW_PUR_COLS)

['ind_ahor_fin_ult1_NEW_PUR', 'ind_aval_fin_ult1_NEW_PUR', 'ind_cco_fin_ult1_NEW_PUR', 'ind_cder_fin_ult1_NEW_PUR', 'ind_cno_fin_ult1_NEW_PUR', 'ind_ctju_fin_ult1_NEW_PUR', 'ind_ctma_fin_ult1_NEW_PUR', 'ind_ctop_fin_ult1_NEW_PUR', 'ind_ctpp_fin_ult1_NEW_PUR', 'ind_deco_fin_ult1_NEW_PUR', 'ind_deme_fin_ult1_NEW_PUR', 'ind_dela_fin_ult1_NEW_PUR', 'ind_ecue_fin_ult1_NEW_PUR', 'ind_fond_fin_ult1_NEW_PUR', 'ind_hip_fin_ult1_NEW_PUR', 'ind_plan_fin_ult1_NEW_PUR', 'ind_pres_fin_ult1_NEW_PUR', 'ind_reca_fin_ult1_NEW_PUR', 'ind_tjcr_fin_ult1_NEW_PUR', 'ind_valo_fin_ult1_NEW_PUR', 'ind_viv_fin_ult1_NEW_PUR', 'ind_nomina_ult1_NEW_PUR', 'ind_nom_pens_ult1_NEW_PUR', 'ind_recibo_ult1_NEW_PUR']


In [14]:
PUR_CANCEL_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1_PUR_OR_CANCEL$", col)]
print(PUR_CANCEL_COLS)

['ind_ahor_fin_ult1_PUR_OR_CANCEL', 'ind_aval_fin_ult1_PUR_OR_CANCEL', 'ind_cco_fin_ult1_PUR_OR_CANCEL', 'ind_cder_fin_ult1_PUR_OR_CANCEL', 'ind_cno_fin_ult1_PUR_OR_CANCEL', 'ind_ctju_fin_ult1_PUR_OR_CANCEL', 'ind_ctma_fin_ult1_PUR_OR_CANCEL', 'ind_ctop_fin_ult1_PUR_OR_CANCEL', 'ind_ctpp_fin_ult1_PUR_OR_CANCEL', 'ind_deco_fin_ult1_PUR_OR_CANCEL', 'ind_deme_fin_ult1_PUR_OR_CANCEL', 'ind_dela_fin_ult1_PUR_OR_CANCEL', 'ind_ecue_fin_ult1_PUR_OR_CANCEL', 'ind_fond_fin_ult1_PUR_OR_CANCEL', 'ind_hip_fin_ult1_PUR_OR_CANCEL', 'ind_plan_fin_ult1_PUR_OR_CANCEL', 'ind_pres_fin_ult1_PUR_OR_CANCEL', 'ind_reca_fin_ult1_PUR_OR_CANCEL', 'ind_tjcr_fin_ult1_PUR_OR_CANCEL', 'ind_valo_fin_ult1_PUR_OR_CANCEL', 'ind_viv_fin_ult1_PUR_OR_CANCEL', 'ind_nomina_ult1_PUR_OR_CANCEL', 'ind_nom_pens_ult1_PUR_OR_CANCEL', 'ind_recibo_ult1_PUR_OR_CANCEL']


In [15]:
DEMOG_COLS = [col for col in df_train.columns 
    if col not in PROD_COLS + NEW_PUR_COLS + PUR_CANCEL_COLS + ["fecha_dato", "ncodpers", "TOTAL_PRODS"]]
print(DEMOG_COLS)

['pais_residencia', 'sexo', 'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall', 'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento']


In [16]:
DEMOG_LAG_COLS = ["ind_nuevo", "antiguedad", "indrel", "tiprel_1mes", 
            "ind_actividad_cliente", "renta", "segmento", "TOTAL_PRODS"]

LAG_COLS = PROD_COLS + PUR_CANCEL_COLS + DEMOG_LAG_COLS
LAG_COLS

['ind_ahor_fin_ult1',
 'ind_aval_fin_ult1',
 'ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ind_nomina_ult1',
 'ind_nom_pens_ult1',
 'ind_recibo_ult1',
 'ind_ahor_fin_ult1_PUR_OR_CANCEL',
 'ind_aval_fin_ult1_PUR_OR_CANCEL',
 'ind_cco_fin_ult1_PUR_OR_CANCEL',
 'ind_cder_fin_ult1_PUR_OR_CANCEL',
 'ind_cno_fin_ult1_PUR_OR_CANCEL',
 'ind_ctju_fin_ult1_PUR_OR_CANCEL',
 'ind_ctma_fin_ult1_PUR_OR_CANCEL',
 'ind_ctop_fin_ult1_PUR_OR_CANCEL',
 'ind_ctpp_fin_ult1_PUR_OR_CANCEL',
 'ind_deco_fin_ult1_PUR_OR_CANCEL',
 'ind_deme_fin_ult1_PUR_OR_CANCEL',
 'ind_dela_fin_ult1_PUR_OR_CANCEL',
 'ind_ecue_fin_ult1_PUR_OR_CANCEL',
 'ind_fond_fin

In [17]:
def extract_subset(df, row_filter, cols):
    return df.loc[row_filter, cols]



def suffixing_cols(df, suffix="_LAG", exclude_cols=["ncodpers"]):
    new_cols = [col + suffix if col not in exclude_cols else col for col in df.columns]
    df.columns = new_cols 
    return df



def extract_by_timestamp_newpur(df, timestamp, sel_prod_cols, 
                                new_pur_suffix="_NEW_PUR", 
                                sel_cols=DEMOG_COLS):
    """extract rows by timestamp and new purchase columns"""
    df = df.copy()
    
    if "ncodpers" not in sel_cols:
        sel_cols = ["ncodpers"] + sel_cols
    
    extracted_dfs = []
    targets = []
    for prod_col in sel_prod_cols:
        pur_col = prod_col + new_pur_suffix
        row_filter = (df["fecha_dato"] == timestamp) & (df[pur_col] == 1)
        
        sub_df = extract_subset(df, row_filter, sel_cols)
        #print("sub_df.shape:", sub_df.shape)
        extracted_dfs.append(sub_df)
        
        ys = [prod_col] * sub_df.shape[0]
        targets.extend(ys)
    
    extracted_dfs = pd.concat(extracted_dfs, axis=0).reset_index(drop=True)
    extracted_dfs["TARGET"] = targets
    
    extracted_dfs = extracted_dfs.sort_values(by="ncodpers")
    return extracted_dfs


def extract_by_timestamp_custid(df, timestamp, customer_ids, sel_cols):
    df = df.copy()
    
    customer_ids = np.unique(customer_ids)
    
    if "ncodpers" not in sel_cols:
        sel_cols = ["ncodpers"] + sel_cols
    #print("sel_cols:", sel_cols)
    
    row_filter = (df["fecha_dato"] == timestamp) & df["ncodpers"].isin(customer_ids)
    df_out = extract_subset(df, row_filter, sel_cols)
    return df_out


def extract_X_y_train(df, timestamp, timestamp_lags, sel_prod_cols,
                      demog_cols=DEMOG_COLS, lag_cols=LAG_COLS):
    out_df = extract_by_timestamp_newpur(df, timestamp, sel_prod_cols, sel_cols=demog_cols)
    
    customer_ids = out_df["ncodpers"]
    for t, lag in enumerate(timestamp_lags):
        assert pd.to_datetime(lag) < pd.to_datetime(timestamp), lag + " lag is not before timestamp " +  timestamp
        lag_label = "_LAG%d" % (t + 1)
        
        lag_df = extract_by_timestamp_custid(df, lag, customer_ids=customer_ids, sel_cols=lag_cols)
        lag_df = suffixing_cols(lag_df, lag_label, exclude_cols=["ncodpers"])
        
        print(lag, lag_label, lag_df.shape)
        
        out_df = out_df.merge(lag_df, how="left", on="ncodpers")
    return out_df


def extract_X_test(train, test, timestamp, timestamp_lags, demog_cols=DEMOG_COLS, lag_cols=LAG_COLS):
    customer_ids = test["ncodpers"]
    out_df = extract_by_timestamp_custid(test, timestamp, customer_ids=customer_ids, sel_cols=demog_cols)
    
    for t, lag in enumerate(timestamp_lags):
        assert pd.to_datetime(lag) < pd.to_datetime(timestamp), lag + " lag is not before timestamp " +  timestamp
        lag_label = "_LAG%d" % (t + 1)
        
        lag_df = extract_by_timestamp_custid(train, lag, customer_ids=customer_ids, sel_cols=lag_cols)
        lag_df = suffixing_cols(lag_df, lag_label, exclude_cols=["ncodpers"])
        
        print(lag, lag_label, lag_df.shape)
        
        out_df = out_df.merge(lag_df, how="left", on="ncodpers")
    return out_df

In [18]:
def most_popular_purchase(df, timestamps, pur_cols, suffix="_NEW_PUR", threshold=None):
    assert isinstance(timestamps, list), "timestamps must be a list"
    prod_popul = df.loc[df["fecha_dato"].isin(timestamps), pur_cols].sum(axis=0)
    prod_popul = prod_popul.sort_values(ascending=False)
    prod_popul.index = [idx.replace(suffix, "") for idx in prod_popul.index]
    prod_popul = prod_popul[prod_popul > 0]
    
    if threshold is None:
        index = prod_popul.index
    else:
        index = prod_popul.index[:threshold]
    return prod_popul[index]

In [19]:
df_train["fecha_dato"].unique()

array(['2015-01-28T00:00:00.000000000', '2015-02-28T00:00:00.000000000',
       '2015-03-28T00:00:00.000000000', '2015-04-28T00:00:00.000000000',
       '2015-05-28T00:00:00.000000000', '2015-06-28T00:00:00.000000000',
       '2015-07-28T00:00:00.000000000', '2015-08-28T00:00:00.000000000',
       '2015-09-28T00:00:00.000000000', '2015-10-28T00:00:00.000000000',
       '2015-11-28T00:00:00.000000000', '2015-12-28T00:00:00.000000000',
       '2016-01-28T00:00:00.000000000', '2016-02-28T00:00:00.000000000',
       '2016-03-28T00:00:00.000000000', '2016-04-28T00:00:00.000000000',
       '2016-05-28T00:00:00.000000000'], dtype='datetime64[ns]')

# Lags up to 6 months

## Extract `X_test` at `2016_06`

In [20]:
timestamp = "2016-06-28"
timestamp_lags = ["2016-05-28", "2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28",
                  "2015-12-28"]

X_test = extract_X_test(df_train, df_test, timestamp, timestamp_lags)
X_test.shape

2016-05-28 _LAG1 (929615, 57)
2016-04-28 _LAG2 (925252, 57)
2016-03-28 _LAG3 (920975, 57)
2016-02-28 _LAG4 (915679, 57)
2016-01-28 _LAG5 (909885, 57)
2015-12-28 _LAG6 (903429, 57)


(929615, 354)

In [21]:
X_test.to_csv(os.path.join(OUT_DIR1, "X_test.csv"), index=False)

## Extract `X` and `y` for training set from `2015-07` to `2016-04`

In [22]:
timestamps = ["2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28", 
              "2015-12-28", "2015-11-28", "2015-10-28", "2015-09-28", 
              "2015-08-28", "2015-07-28"]


lags = {}
lags["2016-04-28"] = ["2016-03-28", "2016-02-28", "2016-01-28", "2015-12-28", "2015-11-28", "2015-10-28"]

lags["2016-03-28"] = ["2016-02-28", "2016-01-28", "2015-12-28", "2015-11-28", "2015-10-28", "2015-09-28"]

lags["2016-02-28"] = ["2016-01-28", "2015-12-28", "2015-11-28", "2015-10-28", "2015-09-28", "2015-08-28"]

lags["2016-01-28"] = ["2015-12-28", "2015-11-28", "2015-10-28", "2015-09-28", "2015-08-28", "2015-07-28"]

lags["2015-12-28"] = ["2015-11-28", "2015-10-28", "2015-09-28", "2015-08-28", "2015-07-28", "2015-06-28"]

lags["2015-11-28"] = ["2015-10-28", "2015-09-28", "2015-08-28", "2015-07-28", "2015-06-28", "2015-05-28"]

lags["2015-10-28"] = ["2015-09-28", "2015-08-28", "2015-07-28", "2015-06-28", "2015-05-28", "2015-04-28"]

lags["2015-09-28"] = ["2015-08-28", "2015-07-28", "2015-06-28", "2015-05-28", "2015-04-28", "2015-03-28"]

lags["2015-08-28"] = ["2015-07-28", "2015-06-28", "2015-05-28", "2015-04-28", "2015-03-28", "2015-02-28"]

lags["2015-07-28"] = ["2015-06-28", "2015-05-28", "2015-04-28", "2015-03-28", "2015-02-28", "2015-01-28"]

In [23]:
popul_pur = most_popular_purchase(df_train, timestamps, NEW_PUR_COLS)
print(len(popul_pur))
print(popul_pur)

23
ind_recibo_ult1      101135
ind_nom_pens_ult1     51994
ind_cco_fin_ult1      50031
ind_nomina_ult1       47049
ind_tjcr_fin_ult1     43012
ind_cno_fin_ult1      25382
ind_ecue_fin_ult1     16469
ind_dela_fin_ult1      8494
ind_ctma_fin_ult1      5271
ind_reca_fin_ult1      4378
ind_valo_fin_ult1      3655
ind_ctop_fin_ult1      2335
ind_fond_fin_ult1      1479
ind_ctpp_fin_ult1      1461
ind_deco_fin_ult1      1451
ind_plan_fin_ult1       483
ind_ctju_fin_ult1       396
ind_deme_fin_ult1       110
ind_pres_fin_ult1        83
ind_cder_fin_ult1        76
ind_hip_fin_ult1         43
ind_viv_fin_ult1         39
ind_aval_fin_ult1         1
dtype: int64


In [24]:
# take 22 most popular product
Y_LABES = popul_pur[:22].index
Y_LABES = list(Y_LABES)
print(len(Y_LABES))
print(Y_LABES)

22
['ind_recibo_ult1', 'ind_nom_pens_ult1', 'ind_cco_fin_ult1', 'ind_nomina_ult1', 'ind_tjcr_fin_ult1', 'ind_cno_fin_ult1', 'ind_ecue_fin_ult1', 'ind_dela_fin_ult1', 'ind_ctma_fin_ult1', 'ind_reca_fin_ult1', 'ind_valo_fin_ult1', 'ind_ctop_fin_ult1', 'ind_fond_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_plan_fin_ult1', 'ind_ctju_fin_ult1', 'ind_deme_fin_ult1', 'ind_pres_fin_ult1', 'ind_cder_fin_ult1', 'ind_hip_fin_ult1', 'ind_viv_fin_ult1']


In [25]:
X_y_train = []
for timestamp in timestamps:
    print("Extract X and y for " + timestamp)
    timestamp_lags = lags[timestamp]
    
    X_y = extract_X_y_train(df_train, timestamp, timestamp_lags, Y_LABES)
    print("X_y.shape", X_y.shape)
    print("X_y.isnull().sum().sum()", X_y.isnull().sum().sum())
    print("")
    X_y_train.append(X_y)

X_y_train = pd.concat(X_y_train, axis=0)
print("X_y_train.shape", X_y_train.shape)
print("X_y_train.isnull().sum().sum()", X_y_train.isnull().sum().sum())

Extract X and y for 2016-04-28
2016-03-28 _LAG1 (26736, 57)
2016-02-28 _LAG2 (25051, 57)
2016-01-28 _LAG3 (24255, 57)
2015-12-28 _LAG4 (23556, 57)
2015-11-28 _LAG5 (22805, 57)
2015-10-28 _LAG6 (22308, 57)
X_y.shape (33022, 355)
X_y.isnull().sum().sum() 1156176

Extract X and y for 2016-03-28
2016-02-28 _LAG1 (27735, 57)
2016-01-28 _LAG2 (25523, 57)
2015-12-28 _LAG3 (24550, 57)
2015-11-28 _LAG4 (23965, 57)
2015-10-28 _LAG5 (23279, 57)
2015-09-28 _LAG6 (22699, 57)
X_y.shape (35258, 355)
X_y.isnull().sum().sum() 1357608

Extract X and y for 2016-02-28
2016-01-28 _LAG1 (36071, 57)
2015-12-28 _LAG2 (33896, 57)
2015-11-28 _LAG3 (33001, 57)
2015-10-28 _LAG4 (32247, 57)
2015-09-28 _LAG5 (31246, 57)
2015-08-28 _LAG6 (30582, 57)
X_y.shape (49072, 355)
X_y.isnull().sum().sum() 1417920

Extract X and y for 2016-01-28
2015-12-28 _LAG1 (26268, 57)
2015-11-28 _LAG2 (24517, 57)
2015-10-28 _LAG3 (23657, 57)
2015-09-28 _LAG4 (22854, 57)
2015-08-28 _LAG5 (22040, 57)
2015-07-28 _LAG6 (21610, 57)
X_y.shape

In [26]:
X_y_train.to_csv(os.path.join(OUT_DIR1, "X_y_train.csv"), index=False)

## Extract `X` and `y` for validation set at `2016_05`

In [27]:
timestamp = "2016-05-28"
timestamp_lags = ["2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28", "2015-12-28", "2015-11-28"]

X_y_val = extract_X_y_train(df_train, timestamp, timestamp_lags, Y_LABES)
X_y_val.shape

2016-04-28 _LAG1 (27874, 57)
2016-03-28 _LAG2 (26371, 57)
2016-02-28 _LAG3 (25691, 57)
2016-01-28 _LAG4 (24935, 57)
2015-12-28 _LAG5 (23967, 57)
2015-11-28 _LAG6 (23552, 57)


(35887, 355)

In [28]:
X_y_val.to_csv(os.path.join(OUT_DIR1, "X_y_val.csv"), index=False)

# Lags up to 3 months

## Extract `X_test` at `2016_06`

In [33]:
timestamp = "2016-06-28"
timestamp_lags = ["2016-05-28", "2016-04-28", "2016-03-28"]

X_test = extract_X_test(df_train, df_test, timestamp, timestamp_lags)
X_test.shape

2016-05-28 _LAG1 (929615, 57)
2016-04-28 _LAG2 (925252, 57)
2016-03-28 _LAG3 (920975, 57)


(929615, 186)

In [34]:
X_test.to_csv(os.path.join(OUT_DIR2, "X_test.csv"), index=False)

## Extract `X` and `y` for training set from `2015-04` to `2016-04`

In [35]:
timestamps = ["2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28", 
              "2015-12-28", "2015-11-28", "2015-10-28", "2015-09-28", 
              "2015-08-28", "2015-07-28", "2015-06-28", "2015-05-28", "2015-04-28"]


lags = {}
lags["2016-04-28"] = ["2016-03-28", "2016-02-28", "2016-01-28"]

lags["2016-03-28"] = ["2016-02-28", "2016-01-28", "2015-12-28"]

lags["2016-02-28"] = ["2016-01-28", "2015-12-28", "2015-11-28"]

lags["2016-01-28"] = ["2015-12-28", "2015-11-28", "2015-10-28"]

lags["2015-12-28"] = ["2015-11-28", "2015-10-28", "2015-09-28"]

lags["2015-11-28"] = ["2015-10-28", "2015-09-28", "2015-08-28"]

lags["2015-10-28"] = ["2015-09-28", "2015-08-28", "2015-07-28"]

lags["2015-09-28"] = ["2015-08-28", "2015-07-28", "2015-06-28"]

lags["2015-08-28"] = ["2015-07-28", "2015-06-28", "2015-05-28"]

lags["2015-07-28"] = ["2015-06-28", "2015-05-28", "2015-04-28"]

lags["2015-06-28"] = ["2015-05-28", "2015-04-28", "2015-03-28"]

lags["2015-05-28"] = ["2015-04-28", "2015-03-28", "2015-02-28"]

lags["2015-04-28"] = ["2015-03-28", "2015-02-28", "2015-01-28"]

In [36]:
popul_pur = most_popular_purchase(df_train, timestamps, NEW_PUR_COLS)
print(len(popul_pur))
print(popul_pur)

23
ind_recibo_ult1      125617
ind_nom_pens_ult1     69125
ind_cco_fin_ult1      61631
ind_nomina_ult1       58173
ind_tjcr_fin_ult1     56491
ind_cno_fin_ult1      31165
ind_ecue_fin_ult1     20599
ind_dela_fin_ult1     11117
ind_reca_fin_ult1      8525
ind_ctma_fin_ult1      5907
ind_valo_fin_ult1      4203
ind_ctop_fin_ult1      3122
ind_fond_fin_ult1      2617
ind_deco_fin_ult1      2244
ind_ctpp_fin_ult1      1952
ind_plan_fin_ult1       545
ind_ctju_fin_ult1       425
ind_deme_fin_ult1       194
ind_pres_fin_ult1       119
ind_cder_fin_ult1       104
ind_hip_fin_ult1         60
ind_viv_fin_ult1         51
ind_aval_fin_ult1         2
dtype: int64


In [37]:
# take 22 most popular product
Y_LABES = popul_pur[:22].index
Y_LABES = list(Y_LABES)
print(len(Y_LABES))
print(Y_LABES)

22
['ind_recibo_ult1', 'ind_nom_pens_ult1', 'ind_cco_fin_ult1', 'ind_nomina_ult1', 'ind_tjcr_fin_ult1', 'ind_cno_fin_ult1', 'ind_ecue_fin_ult1', 'ind_dela_fin_ult1', 'ind_reca_fin_ult1', 'ind_ctma_fin_ult1', 'ind_valo_fin_ult1', 'ind_ctop_fin_ult1', 'ind_fond_fin_ult1', 'ind_deco_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_plan_fin_ult1', 'ind_ctju_fin_ult1', 'ind_deme_fin_ult1', 'ind_pres_fin_ult1', 'ind_cder_fin_ult1', 'ind_hip_fin_ult1', 'ind_viv_fin_ult1']


In [38]:
X_y_train = []
for timestamp in timestamps:
    print("Extract X and y for " + timestamp)
    timestamp_lags = lags[timestamp]
    
    X_y = extract_X_y_train(df_train, timestamp, timestamp_lags, Y_LABES)
    print("X_y.shape", X_y.shape)
    print("X_y.isnull().sum().sum()", X_y.isnull().sum().sum())
    print("")
    X_y_train.append(X_y)

X_y_train = pd.concat(X_y_train, axis=0)
print("X_y_train.shape", X_y_train.shape)
print("X_y_train.isnull().sum().sum()", X_y_train.isnull().sum().sum())

Extract X and y for 2016-04-28
2016-03-28 _LAG1 (26736, 57)
2016-02-28 _LAG2 (25051, 57)
2016-01-28 _LAG3 (24255, 57)
X_y.shape (33022, 187)
X_y.isnull().sum().sum() 313376

Extract X and y for 2016-03-28
2016-02-28 _LAG1 (27735, 57)
2016-01-28 _LAG2 (25523, 57)
2015-12-28 _LAG3 (24550, 57)
X_y.shape (35258, 187)
X_y.isnull().sum().sum() 393736

Extract X and y for 2016-02-28
2016-01-28 _LAG1 (36071, 57)
2015-12-28 _LAG2 (33896, 57)
2015-11-28 _LAG3 (33001, 57)
X_y.shape (49072, 187)
X_y.isnull().sum().sum() 387408

Extract X and y for 2016-01-28
2015-12-28 _LAG1 (26268, 57)
2015-11-28 _LAG2 (24517, 57)
2015-10-28 _LAG3 (23657, 57)
X_y.shape (31031, 187)
X_y.isnull().sum().sum() 316848

Extract X and y for 2015-12-28
2015-11-28 _LAG1 (33684, 57)
2015-10-28 _LAG2 (31219, 57)
2015-09-28 _LAG3 (30180, 57)
X_y.shape (42385, 187)
X_y.isnull().sum().sum() 439320

Extract X and y for 2015-11-28
2015-10-28 _LAG1 (28036, 57)
2015-09-28 _LAG2 (25122, 57)
2015-08-28 _LAG3 (24120, 57)
X_y.shape (3

In [39]:
X_y_train.to_csv(os.path.join(OUT_DIR2, "X_y_train.csv"), index=False)

## Extract `X` and `y` for validation set at `2016_05`

In [40]:
timestamp = "2016-05-28"
timestamp_lags = ["2016-04-28", "2016-03-28", "2016-02-28"]

X_y_val = extract_X_y_train(df_train, timestamp, timestamp_lags, Y_LABES)
X_y_val.shape

2016-04-28 _LAG1 (27874, 57)
2016-03-28 _LAG2 (26371, 57)
2016-02-28 _LAG3 (25691, 57)


(35887, 187)

In [41]:
X_y_val.to_csv(os.path.join(OUT_DIR2, "X_y_val.csv"), index=False)